In [2]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='aml1p-ml-wus2', subscription_id='48bbc269-ce89-4f6f-9a12-c6f91fcb772d', resource_group='aml1p-rg')

modulefunc = Module.from_yaml(ws, yaml_file='./noop/1in2out.spec.yaml')
data = Dataset.get_by_name(ws, 'training_data')

In [3]:
@dsl.pipeline(
    name='A pipeline composed with nodes 1 in 2 outs',
    description='A sample',
    default_compute_target='windows-cpu'
)
def cell_division():
    layer = 3
    nodes = []
    nodes.append(modulefunc(input1=data))
    for i in range(0, layer-1):
        print('i=', i, ' nodes len=', len(nodes))
        current_layer_nodes = []
        for j in range(0, pow(2,i)):
            print('j=', j)
            n = nodes[-j-1]
            current_layer_nodes.append(modulefunc(input1=n.outputs.output1))
            current_layer_nodes.append(modulefunc(input1=n.outputs.output2))
        nodes = nodes + current_layer_nodes
    
    return {**nodes[-1].outputs}

pipeline = cell_division()

pipeline.validate()

i= 0  nodes len= 1
j= 0
i= 1  nodes len= 3
j= 0
j= 1


ValueError: Target windows-cpu does not exist in the workspace.

In [4]:
pipeline.submit(experiment_name='samples')

windows-cpu not found in workspace, assume this is an AmlCompute
Submitted PipelineRun 54977da9-fa5d-47e5-bab5-f91cebf0201e
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/54977da9-fa5d-47e5-bab5-f91cebf0201e?wsid=/subscriptions/48bbc269-ce89-4f6f-9a12-c6f91fcb772d/resourcegroups/aml1p-rg/workspaces/aml1p-ml-wus2


Experiment,Id,Type,Status,Details Page,Docs Page
samples,54977da9-fa5d-47e5-bab5-f91cebf0201e,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation
